In [119]:
from pymongo import MongoClient

client = MongoClient('localhost:27017')
db = client.imxDatabase
db.drop_collection('proposals')
db.create_collection('proposals',
                     validator={
                         "$jsonSchema": {
                             "bsonType": "object",
                             "required": [ "proposta" ],
                             "properties": {
                                 "proposta": {
                                     "bsonType": "int",
                                     "description": "Numero para controle interno, vinculado ao BD relacional."
                                 }
                             }
                         }
                     })
proposals = db.proposals

In [120]:
import re

In [121]:
def parser_sample_RAfT(path):
    with open(path) as file:
        file_string = file.read()
    list_s = re.split('\n', file_string)
    regex = re.compile(r'(.*) = (.*).*')
    tuples_sample = [(m.group(1), m.group(2)) for l in list_s for m in [regex.search(l)] if m]
    
    dict_file = {}
    for t in tuples_sample:
        m = re.match(r'^#(.*)', t[0])
        if m:
            key = m[1].lower()
        else:
            key = t[0].lower()
        if re.search(r'path|file', key):
            value = t[1]
        else:
            numbers = re.findall(r'-?\d+\.?\d*', t[1])
            if numbers == []:
                value = t[1]
            else:
                numbers = [float(s) for s in numbers]
            if len(numbers) == 1:
                value = numbers[0]
            elif len(numbers) > 1:
                value = {'x': numbers[0], 'y': numbers[1]}
        dict_file[key] = value
    return dict_file

In [122]:
def create_proposal(proposal_id, collection):
    collection.insert_one({'proposta': proposal_id})

In [123]:
def add_experiment(proposal_id, collection, **kwargs):
    collection.update_one({'proposta': proposal_id}, { '$push': { 'experimentos': kwargs } } )

In [128]:
def choose_experiment(experiments):
    if len(experiments) > 1:
        n = 1
        print('There are more than one experiment in this proposal.') 
        for experiment in experiments:
            print('Experiment ', n, ',')
            for key, value in experiment.items():
                if key != 'amostras':
                    print('\t', key, ' : ', value)
            n += 1
        return input('Please choose one to add the sample ')
    else:
        return '0'

In [129]:
def choose_sample(samples):
    if len(samples) > 1:
        n = 1
        print('There are more than one sample in this experiment.') 
        for sample in samples:
            print('Sample ', n, ',')
            for key, value in samples.items():
                if key != 'medidas':
                    print('\t', key, ' : ', value)
            n += 1
        return input('Please choose one to add the measurement')
    else:
        return '0'

In [130]:
def add_sample(proposal_id, collection, **kwargs):
    proposal = collection.find({'proposta': proposal_id})[0]
    idx = choose_experiment(proposal['experimentos'])
                    
    sample_to_add = 'experimentos.' + idx + '.amostras'
    collection.update_one({'proposta': proposal_id}, { '$push': { sample_to_add: kwargs } } )

In [ ]:
def add_measurements(proposal_id, collection, sample_path, RAfT_path):
    proposal = collection.find({'proposta': proposal_id})[0]
    num_experiments = len(proposal['experimentos'][])
                    
    sample_to_add = 'experimentos.' + idx + '.amostras'
    collection.update_one({'proposta': proposal_id}, { '$push': { sample_to_add: kwargs } } )    

In [131]:
create_proposal(20180001, proposals)

In [132]:
args = {
    "nome": "Análise da calcificação em ossos",
    "descricao": "Analisar a qualidade da calcificação em ossos tratados com esferas de cálcio de diferentes tamanhos de poros."
}


add_experiment(20180001, proposals, **args)

In [95]:
import pprint
for e in proposals.find({'proposta': 20180001})[0]['experimentos']:
    for key, value in e.items():
        print(key)

nome
descricao


In [96]:
a = input('isert something')

isert something1


In [133]:
args = {
    "nome":"Osso tratado com esferas contendo poros de 9um.",
    "descricao":"Osso tratado após a inserção de esferas de cálcio com poros de 9um e com 7 meses de tratamento.",
    "contrastante":"Iodo",
    "Tempo de Cura":"20 horas",
    "Aplicação":"Absorção"
}
add_sample(20180001, proposals, **args)

In [106]:
proposals.update_one({'proposta': 20180001}, { '$push': { 'experimentos.0.amostras': {'a':1} } } )

In [134]:
pprint.pprint(proposals.find_one())

{'_id': ObjectId('5c072a24b0351606ee7957ef'),
 'experimentos': [{'amostras': [{'Aplicação': 'Absorção',
                                 'Tempo de Cura': '20 horas',
                                 'contrastante': 'Iodo',
                                 'descricao': 'Osso tratado após a inserção de '
                                              'esferas de cálcio com poros de '
                                              '9um e com 7 meses de '
                                              'tratamento.',
                                 'nome': 'Osso tratado com esferas contendo '
                                         'poros de 9um.'}],
                   'descricao': 'Analisar a qualidade da calcificação em ossos '
                                'tratados com esferas de cálcio de diferentes '
                                'tamanhos de poros.',
                   'nome': 'Análise da calcificação em ossos'}],
 'proposta': 20180001}
